In [ ]:
# Author : Jonas Rashidi

import cv2
import numpy as np
from PIL import Image
import os

def upscale_image(image, scale_percent=150):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    return cv2.resize(image, dim, interpolation=cv2.INTER_CUBIC)

def remove_red_lines_and_blur(input_path, output_path, upscale_factor=150):
    if not os.path.exists(input_path):
        print(f"File not found: {input_path}")
        return
    
    img_colored = cv2.imread(input_path, cv2.IMREAD_COLOR)
    
    if img_colored is None:
        print(f"Failed to load image from: {input_path}")
        return
    
    img_colored_upscaled = upscale_image(img_colored, upscale_factor)
    img_hsv = cv2.cvtColor(img_colored_upscaled, cv2.COLOR_BGR2HSV)

    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(img_hsv, lower_red2, upper_red2)

    red_mask = mask1 | mask2
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    refined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel, iterations=3)
    refined_mask = cv2.dilate(refined_mask, kernel, iterations=1)

    inpainted_image = cv2.inpaint(img_colored_upscaled, refined_mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)
    for i in range(3):
        inpainted_image = cv2.inpaint(inpainted_image, refined_mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)

    for i in range(1, 4):
        inpainted_image = cv2.inpaint(inpainted_image, refined_mask, inpaintRadius=i, flags=cv2.INPAINT_NS)

    denoised_image = cv2.fastNlMeansDenoisingColored(inpainted_image, None, 10, 10, 7, 21)
    gray_image = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)
    _, detail_mask = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    detail_mask = cv2.cvtColor(detail_mask, cv2.COLOR_GRAY2BGR)

    final_image = cv2.addWeighted(denoised_image, 0.8, detail_mask, 0.2, 0)
    result_img = Image.fromarray(np.uint8(final_image))
    result_img.save(output_path, format="PNG", quality=100)

    print(f"Image saved to: {output_path}")

input_path = r"C:\Users\User\Downloads\Input-F cells.jpeg"
output_path = r'C:\Users\User\Downloads\output_image_no_blur2_final_v5.png'
remove_red_lines_and_blur(input_path, output_path)
